In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.constraints import max_norm
from keras import backend as K
from scipy import interpolate

np.set_printoptions(precision=3, suppress=True)

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def build_and_compile_model(norm):
    
  model = keras.Sequential([

      norm,
      layers.Dropout(.3),      
      layers.Dense(64, activation='relu'),
      layers.Dropout(.2),
      layers.Dense(64, activation='relu'),
      layers.Dropout(.2),
      layers.Dense(1, kernel_constraint=tf.keras.constraints.NonNeg())
      
  ])
  
  model.compile(loss='mean_squared_error',
                optimizer=tf.keras.optimizers.Adam(0.001),
                metrics=[coeff_determination])
  return model

In [ ]:
raw_dataset=pd.read_excel("input.xlsx", sheet_name="rilem")
dataset=raw_dataset.copy()
df_master=pd.read_excel("input.xlsx", sheet_name="master(av_correct)")

promenljive=["Temperature (°C)", "Air voids (%) ", "ITS (MPa)", "Binder content (%)"]
dataset=dataset[promenljive]
df_master=df_master[promenljive]


train_dataset=dataset
test_dataset=df_master

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop("ITS (MPa)")
test_labels = test_features.pop("ITS (MPa)")

normalizer = tf.keras.layers.experimental.preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))
first=np.array(train_features[:1])

#---------#

dnn_model = build_and_compile_model(normalizer)

In [ ]:
%%time
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=100)
history = dnn_model.fit(
    train_features,
    train_labels,
    validation_data=(test_features,test_labels),
    verbose=0, epochs=49, callbacks=[es])

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()
train_preditions = dnn_model.predict(train_features).flatten()

print(r2_score(test_labels, test_predictions),"za validaciju")
print(r2_score(train_labels, train_preditions),"za trening")

In [ ]:
plt.figure(figsize=(9,3))
sns.set(rc={"figure.dpi":300, 'savefig.dpi':300},font="Times New Roman")

x=[]
for i in range(len(history.history['val_loss'])):
    x.append(i)

sns.lineplot(x=x,y=history.history['loss'], label='Сет за обучавање')
sns.lineplot(x=x,y=history.history['val_loss'], label='Сет за валидацију')
plt.ylim([0, 1])
plt.xlim([0, 49])
plt.xlabel('Епоха')
plt.ylabel('Грешка [ITS]')

In [ ]:
min(history.history['loss'])

In [ ]:
test_results = {}
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)
#print(test_results)

In [ ]:
#pd.DataFrame(test_results, index=['Mean squared error [ITS]',"Rsq"]).T

In [ ]:
plt.figure(figsize=(5,5))
sns.set(rc={"figure.dpi":300, 'savefig.dpi':300},font="Times New Roman")

sns.scatterplot(x=test_labels, y=test_predictions, marker="X", color=sns.color_palette("deep")[1], s=100)
plt.plot([.4,1.6],[.4,1.6], lw=1,linestyle='dashed')

plt.xlabel('Оригиналне вредности ITS-а [MPa]')
plt.ylabel('Прогнозиране вредности ITS-а [MPa]')
lims = [.4, 1.6]
plt.xlim(lims)
plt.ylim(lims)
plt.legend(loc='upper left')


In [ ]:
plt.figure(figsize=(5,5))
sns.set(rc={"figure.dpi":300, 'savefig.dpi':300},font="Times New Roman")

sns.kdeplot(x=test_labels, y=test_predictions, shade=True)
sns.scatterplot(x=test_labels, y=test_predictions, marker="X", color=sns.color_palette("deep")[1], s=100)
plt.plot([.4,1.6],[.4,1.6], lw=1,linestyle='dashed')

plt.xlabel('Измерене вредности ITS-а [MPa]')
plt.ylabel('Процењене вредности ITS-а [MPa]')
lims = [.4, 1.6]
plt.xlim(lims)
plt.ylim(lims)

plt.title("r2=0.92", loc="right")

In [ ]:
df_master["ITS - Predicted [MPa]"]=(test_predictions)
#df_master.to_excel("rezultat master validacije.xlsx")
ab_master=df_master.iloc[0:4]
sma_master=df_master.iloc[4:8]
bns_master=df_master.iloc[8:12]

In [ ]:
a=abs(test_labels-test_predictions)

In [ ]:
a.mean()

In [ ]:
test_labels.mean()

In [ ]:
a.mean()/test_labels.mean()

In [ ]:
import shap
# explain the model's predictions using SHAP
explainer = shap.TreeExplainer(dnn_model)
shap_values = explainer.shap_values(train_features)

# visualize the first prediction's explanation 
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0,:], X_train.iloc[0,:])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(max_depth=6, random_state=0, n_estimators=10)
model.fit(train_features, train_labels)

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(train_features)

# visualize the first prediction's explanation 
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0,:], train_features.iloc[0,:])

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, train_features)

In [ ]:
plt.figure(figsize=(9,3))
sns.set(rc={"figure.dpi":300, 'savefig.dpi':300},font="Times New Roman")

hap_values = shap.TreeExplainer(model).shap_values(train_features)
shap.summary_plot(shap_values, train_features, plot_type="bar")

In [ ]:
plt.figure(figsize=(9, 10))
sns.set(rc={"figure.dpi":300, 'savefig.dpi':300},font="Times New Roman")

shap.summary_plot(shap_values, train_features)